# ![](https://ga-dash.s3.amazonaws.com/production/assets/logo-9f88ae6c9c3871690e33280fcf557f33.png) Regression Challenge
Week 6 | Days 4-5

## The Times university ranking dataset analysis

A university has chosen your consulting firm to help them understand and improve their standings in the Times rankings of universities.  Your company will be presentings your findings to a review board from the university Friday afternoon at 4pm.  The review board consists of members from across the university: both professors and administrators.  Your presentation must contain both a technical defense of your work and solid recommendations to move forward.

In this challenge, you will draw on the skills you have learned over the past three weeks to create a model of university prestige using the provided predictors. Specifically, your goal is to **predict the total score for each university for the year 2016**. This score directly maps into the university ranking.

You will be drawing on the following skills:
- Basic python and pandas skills
- Data cleaning
- EDA
- Modeling
    - Regression
    - Regularization
    - kNN
    - SGD
- Cross validation

## The Dataset

The data is in a csv file in your repo. It contains the following columns:

- **world_rank** - world rank for the university. Contains rank ranges and equal ranks (e.g. = 94 and 201-250).
- **university_name** - name of university.
- **country** - country of each university.
- **teaching** - university score for teaching (the learning environment).
- **international** - university score international outlook (staff, students, research).
- **research** - university score for research (volume, income and reputation).
- **citations** - university score for citations (research influence).
- **income** - university score for industry income (knowledge transfer).
- **total_score** - total score for university, used to determine rank.
- **num_students** - number of students at the university.
- **student_staff_ratio** - Number of students divided by number of staff.
- **international_students** - Percentage of students who are international.
- **female_male_ratio** - Female student to Male student ratio.
- **year** - year of the ranking (2011 to 2016 included).

### The target for our model is the **total score**, which directly corresponds to the final ranking.

### Note: A total score reported as "-" should be considered a 0, and '-' entries in the submisison will be scored as  0. Consider the implications of this for calculating the loss (MSE).


## Guidelines

The analysis is up to you. **This is fully open-ended.** You are expected to:

- Load the packages you need to do analysis
- Perform EDA on variables of interest
- Form a hypothesis on what is important for the score
- Check your data for problems, clean and munge data into correct formats
- Create or combine new columns/features where beneficial
- Perform statistical analysis with regression and describe the results

---

We will be here in class to help, but if you do not know how to do something, we expect you to **check documentation first**.

**You are not expected to know how to do things by heart. Knowing how to effectively look up the answers on the internet is a critical skill for data scientists!**

## Deliverables

- You will be provided with the data and targets from 2011 through 2015 and the data (no targets) for 2016.
    + Final answers should be submitted by filling in the predicted values for each university in the **submission sheet**. (.to_csv() should be useful for this). Note there are three columns: Rank, Name, and Score. The row order matches the rankings list from the 2016 data.
    + Your submission will be assessed on MSE for the Score column -- so consider your loss functions! <br><br>


- Your team will also submit a clean and annotated version of the **analysis notebook** you used to make your model and produce predictions. <br><br>


- Finally, you will design and deliver an **8 minute presentation** to the university review board. Your presentation must:
    + Defend your model selection
    + Defend your model performance
    + Interpret the model
    + Give clear guidelines to improve rankings


- Remember your audience contains some experts who will want to understand the strength of your predictions, others who will want to know they are getting the best version possible, and still more who only want to know which actions will help their ranking the most.  Do your best to meet all these demands without alienating anyone.

## Teams

You will be working as part of a team on this:

<table>
<tr>
<th>Team 8d599</th>
<th>Team 5d87e</th>
<th>Team 41cfa</th>
<th>Team c13cf</th>
</tr>
<tr>
<td>Ryan</td>
<td>Yuriy</td>
<td>Ridhi</td>
<td>Andrew</td>
</tr>
<tr>
<td>Arafat</td>
<td>Joe</td>
<td>Alex</td>
<td>Will</td>
</tr>
<tr>
<td>Mark</td>
<td>Eric</td>
<td>Donna</td>
<td>Tim</td>
</tr>
<tr>
<td></td>
<td>Scott</td>
<td></td>
<td>Marina</td>
</tr>
</table>


As a team you are responsible for one copy of each deliverable to be submitted as an issue on this repo by **4 PM Friday**.  That is to say, one each of:
- Submission csv
- Analysis notebook
- 8 min Presentation

Every team member must speak in the final presentation, and be ready to answer questions on the process the group used to refine, choose, and interpret the model used to make predictions.

In [336]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('fivethirtyeight')

from ipywidgets import *
from IPython.display import display

from sklearn.svm import SVC

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from sklearn import svm, linear_model
from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import  cross_val_score, cross_val_predict
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import train_test_split

In [337]:
all_data = pd.read_csv('datasets/challenge-dataset.csv', encoding='latin-1')
data = all_data.loc[all_data['year'] !=2016,: ]

In [338]:
data.head()

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
0,1,Harvard University,United States of America,99.7,72.4,98.7,98.8,34.5,96.1,"20,152",8.9,25%,NaN,2011
1,2,California Institute of Technology,United States of America,97.7,54.6,98.0,99.9,83.7,96,"2,243",6.9,27%,33 : 67,2011
2,3,Massachusetts Institute of Technology,United States of America,97.8,82.3,91.4,99.9,87.5,95.6,"11,074",9.0,33%,37 : 63,2011
3,4,Stanford University,United States of America,98.3,29.5,98.1,99.2,64.3,94.3,"15,596",7.8,22%,42:58:00,2011
4,5,Princeton University,United States of America,90.9,70.3,95.4,99.9,-,94.2,"7,929",8.4,27%,45:55:00,2011


In [339]:
data.describe()

,teaching,research,citations,student_staff_ratio,year
count,1803.000000,1803.000000,1803.000000,1751.000000,1803.000000
mean,40.560233,39.363006,65.186467,18.141348,2013.221852
std,17.976259,21.079805,19.626047,10.933447,1.315252
min,10.700000,6.500000,3.100000,2.600000,2011.000000
25%,27.300000,22.900000,51.150000,11.800000,2012.000000
50%,37.000000,34.600000,65.000000,15.900000,2013.000000
75%,49.750000,50.600000,80.400000,20.700000,2014.000000
max,99.700000,99.400000,100.000000,72.500000,2015.000000


In [340]:
'''Null Columns'''
print (data.columns[data.isnull().sum()>0])

Index(['num_students', 'student_staff_ratio', 'international_students',
       'female_male_ratio'],
      dtype='object')


In [341]:
data.isnull().sum()

world_rank                  0
university_name             0
country                     0
teaching                    0
international               0
research                    0
citations                   0
income                      0
total_score                 0
num_students               52
student_staff_ratio        52
international_students     57
female_male_ratio         172
year                        0
dtype: int64

In [342]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1803 entries, 0 to 1802
Data columns (total 14 columns):
world_rank                1803 non-null object
university_name           1803 non-null object
country                   1803 non-null object
teaching                  1803 non-null float64
international             1803 non-null object
research                  1803 non-null float64
citations                 1803 non-null float64
income                    1803 non-null object
total_score               1803 non-null object
num_students              1751 non-null object
student_staff_ratio       1751 non-null float64
international_students    1746 non-null object
female_male_ratio         1631 non-null object
year                      1803 non-null int64
dtypes: float64(4), int64(1), object(9)
memory usage: 211.3+ KB


In [343]:
'''Checking for Hyphens in the International Columns'''
np.sum([data['international'] == '-'])

9

In [344]:
'''Replacing the Hyphens with the NAN to impute later'''
data['international'] = data.international.replace('-', np.nan)

/Users/ajinthchristudas/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [345]:
data['international'] = data.international.replace('-', np.nan)

from sklearn.preprocessing import Imputer

international_imputer = Imputer(strategy='median')
international_imputer_fit = international_imputer.fit(data['international'].values.reshape(-1, 1))

international_imputer_transform = international_imputer_fit.transform(
    data['international'].values.reshape(-1, 1))

data['international'] = international_imputer_transform

/Users/ajinthchristudas/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/ajinthchristudas/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


In [346]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1803 entries, 0 to 1802
Data columns (total 14 columns):
world_rank                1803 non-null object
university_name           1803 non-null object
country                   1803 non-null object
teaching                  1803 non-null float64
international             1803 non-null float64
research                  1803 non-null float64
citations                 1803 non-null float64
income                    1803 non-null object
total_score               1803 non-null object
num_students              1751 non-null object
student_staff_ratio       1751 non-null float64
international_students    1746 non-null object
female_male_ratio         1631 non-null object
year                      1803 non-null int64
dtypes: float64(5), int64(1), object(8)
memory usage: 211.3+ KB


In [347]:
'''Count of Hyphens in the Total_Score Column'''
np.sum([data['total_score'] == '-'])

802

In [348]:
'''Replacing the Hyphens with the NAN to impute later'''
# data['total_score'] = data.total_score.replace('-', np.nan)

'Replacing the Hyphens with the NAN to impute later'

In [349]:
# data['total_score'] = data.total_score.replace('-', np.nan)

# from sklearn.preprocessing import Imputer

# total_score_imputer = Imputer(strategy='median')
# total_score_imputer_fit = total_score_imputer_fit.fit(data['total_score'].values.reshape(-1, 1))

# total_score_imputer_transform = total_score_imputer_fit.transform(
#     data['total_score'].values.reshape(-1, 1))

# data['total_score'] = total_score_imputer_transform

In [350]:
'''Income Imputer '''
for k, v in data['income'].items():
    if v == '-':
        data.set_value(k, 'income', np.nan)
    else:
        data.set_value(k, 'income', v)
        
data['income'] = data['income'].astype(float)        
        
income_imputer = Imputer(strategy='median')
income_imputer_fit = income_imputer.fit(data['income'].values.reshape(-1, 1))

income_imputer_transform = income_imputer_fit.transform(
    data['international'].values.reshape(-1, 1))

data['income'] = income_imputer_transform

/Users/ajinthchristudas/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/ajinthchristudas/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [351]:
'''Country'''
for k, v in data['country'].items():
    if v == 'United States of America':
        data.set_value(k, 'country', 1)
    else:
        data.set_value(k, 'country', 0)

data['country'] = data['country'].astype(float)

/Users/ajinthchristudas/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [352]:
'''Num Students'''
data['num_students'] = data['num_students'].str.replace(',', '').astype(float)

/Users/ajinthchristudas/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [353]:
data['international_students']=[str(pct).split('%')[0] for pct in data['international_students']]
data['international_students']=[float(pct) for pct in data['international_students']]

/Users/ajinthchristudas/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/ajinthchristudas/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [354]:
'''International Students Imputer'''

data['international_students']=[str(pct).split('%')[0] for pct in data['international_students']]
data['international_students']=[float(pct) for pct in data['international_students']]

data['international_students'] = data.international_students.replace('-', np.nan)
from sklearn.preprocessing import Imputer

international_s_imputer = Imputer(strategy='median')
international_s_imputer_fit = international_s_imputer.fit(data['international_students'].values.reshape(-1, 1))

international_s_imputer_transform = international_s_imputer_fit.transform(
    data['international_students'].values.reshape(-1, 1))

data['international_students'] = international_s_imputer_transform

/Users/ajinthchristudas/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/ajinthchristudas/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/ajinthchristudas/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

In [355]:
# '''Fixing total Score'''
# def change_hyphen(x): 
#     if x == '-': 
#         return float(0)
#     else: 
#         return float(x)
    
# data['total_score'] = data['total_score'].apply(change_hyphen)

In [356]:
# data['total_score'] = data['total_score'].str.replace('-', '0').astype(float)

/Users/ajinthchristudas/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [335]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1803 entries, 0 to 1802
Data columns (total 14 columns):
world_rank                1803 non-null object
university_name           1803 non-null object
country                   1803 non-null object
teaching                  1803 non-null float64
international             1803 non-null object
research                  1803 non-null float64
citations                 1803 non-null float64
income                    1803 non-null object
total_score               1803 non-null object
num_students              1751 non-null object
student_staff_ratio       1751 non-null float64
international_students    1746 non-null object
female_male_ratio         1631 non-null object
year                      1803 non-null int64
dtypes: float64(4), int64(1), object(9)
memory usage: 211.3+ KB
